In [34]:
import tensorflow as tf

In [35]:

tf.test.is_gpu_available()


True

In [36]:
import pandas as pd

file_path3 = 'true.csv'
file_path4 = 'false.csv'

c1 =  pd.read_csv(file_path3)
c2 =  pd.read_csv(file_path4)

c1['BinaryNumTarget'] = 1.0
c2['BinaryNumTarget'] = 0.0

df = pd.concat([c1,c2], ignore_index=True)

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

df.head()

,author,statement,target,manual_keywords,tweet,5_label_majority_answer,3_label_majority_answer,BinaryNumTarget
0,Daniel Funke,At 17 years old Kyle (Rittenhouse) was perfect...,False,"Kyle,possess,rifle,legal",@fattycattie @robquinnpc @SammyTMcCarty @WCCO ...,Mostly Agree,Agree,0.0
1,Jill Terreri,"""I actually in 2005 ran on Medicare for all.""",True,"2005, ran on Medicare","never forget, Matt Santos ran on Medicare for ...",Mostly Agree,Agree,1.0
2,Katie Sanders,"FIFA ""pressured Brazil into passing a so-calle...",True,"FIFA, budweiser bill",@globeandmail @Globe_Health #Budweiser and #Fi...,Mostly Agree,Agree,1.0
3,Manuela Tobias,"""You know what Amazon paid in federal income t...",True,"Amazon, federal income taxes, zero",@SenSanders The working class is literally bei...,Mostly Agree,Agree,1.0
4,Katie Sanders,Obama promised to cut the deficit by half by t...,True,"Obama, promised, cut deficit",@CarlSpoerer2018 @TaraTran888 @BLEDSTATE_RED O...,Mostly Agree,Agree,1.0


In [37]:





# Load the dataset
file_path = 'Truth_Seeker_Model_Dataset.csv'
file_path2 = 'Features_For_Traditional_ML_Techniques.csv'
file_path3 = 'Downloads/Truth_Seeker_Model_Dataset_With_TimeStamps 1.xlsx'
df = pd.read_csv(file_path)

df2 = pd.read_csv(file_path2)

df3 = pd.read_excel(file_path3)

#df = df.drop(columns=["Unnamed: 0"])

df2 = df2.drop(columns=["Unnamed: 0"])

df3 = df3.drop(columns=["Unnamed: 0"])


In [38]:

#df = df.sample(frac=1, random_state=42).reset_index(drop=True)


In [39]:


import warnings
warnings.filterwarnings("ignore")



In [40]:


#from transformers import *  # this is HuggingFace library
#import tensorflow as tf
#from transformers import BertTokenizer, TFBertModel, BertConfig

from transformers import DistilBertTokenizer, DistilBertConfig, TFDistilBertModel


In [41]:

# this will download the BERT Tokenizer
#bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")  
# this will download the BERT Trained Model
# output_hidden_states=False, as we are training & not interested in output state.
#config = BertConfig.from_pretrained("bert-base-uncased",output_hidden_states=False) # dropout=0.2, attention_dropout=0.2
#bert_model = TFBertModel.from_pretrained('bert-base-uncased', config=config)


# Download the DistilBERT Tokenizer
distilbert_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Download the DistilBERT Trained Model
# output_hidden_states=False is the default for DistilBert, so it's not strictly necessary to set it.
distilbert_config = DistilBertConfig.from_pretrained("distilbert-base-uncased") # You can add config parameters here if needed
distilbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=distilbert_config)



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [42]:


import re
from tqdm import tqdm


def clean_text(temp):
    temp=re.sub("@\S+", " ", temp)
    temp=re.sub("https*\S+", " ", temp)
    temp=re.sub("#\S+", " ", temp)
    temp=re.sub("\'\w+", '', temp)
    temp=re.sub(r'\w*\d+\w*', '', temp)
    temp=re.sub('\s{2,}', " ", temp)
   
    return temp.strip()

df['tweet_clean'] = df['tweet'].apply(clean_text)
df['stat'] = df['statement'].apply(clean_text)
df['auth'] = df['author'].apply(clean_text)
df['mank'] = df['manual_keywords'].apply(clean_text)
df['fivlab'] = df['5_label_majority_answer'].apply(clean_text)
df['thrlab'] = df['3_label_majority_answer'].apply(clean_text)
#sentences = df['tweet_clean'] + " "+ df['stat'] + " " + df['auth'] + " " + df['mank'] + " " + df['fivlab'] + " " + df['thrlab'] 

sentences = df['tweet_clean'] + " "+ df['stat'] + " " + df['auth'] + " " + df['mank']


In [43]:



import numpy as np


input_ids=[]
attention_masks=[]

for sent in tqdm(sentences):
    bert_inp=distilbert_tokenizer.encode_plus(sent,add_special_tokens = True,truncation=True,max_length =28,pad_to_max_length = True,return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
target = np.array(df['BinaryNumTarget'])

100%|████████████████████████████████████████████████████████████████████████| 134198/134198 [01:49<00:00, 1227.73it/s]


In [44]:

distilbert_tokenizer.convert_ids_to_tokens(101)



'[CLS]'

In [45]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test,train_mask,test_mask=train_test_split(input_ids,target,attention_masks,test_size=0.2,random_state=42)

In [46]:


distilbert_model.summary()

Model: "tf_distil_bert_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
Total params: 66,362,880
Trainable params: 66,362,880
Non-trainable params: 0
_________________________________________________________________


In [47]:


from tensorflow.keras.layers import Input, Embedding, SpatialDropout1D, LSTM, Dense, Dropout, Flatten, Concatenate, GlobalMaxPool1D , Bidirectional
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
import numpy as np

# Textual Input
# text_input = Input(shape=(max_sequence_length,), name='text_input')
# embedding = Embedding(input_dim=max_num_words, output_dim=embedding_dim, input_length=max_sequence_length)(text_input)
input_ids_in = tf.keras.layers.Input(shape=(28,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(28,), name='masked_token', dtype='int32') 
embedding_layer = distilbert_model(input_ids_in, attention_mask=input_masks_in)[0] 
lstm = Bidirectional(LSTM(8, return_sequences=True, dropout=0.1, recurrent_dropout=0))(embedding_layer)
pooling = GlobalMaxPool1D()(lstm)
X = Dense(8, activation='relu')(pooling)
X = Dropout(0.2)(X)
X = Dense(1, activation='sigmoid')(X)

model = Model(inputs=[input_ids_in, input_masks_in], outputs=X)


for layer in distilbert_model.layers[:3]:  
    layer.trainable = False

model.summary()




Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 28)]         0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 28)]         0           []                               
                                                                                                  
 tf_distil_bert_model_1 (TFDist  TFBaseModelOutput(l  66362880   ['input_token[0][0]',            
 ilBertModel)                   ast_hidden_state=(N               'masked_token[0][0]']           
                                one, 28, 768),                                                    
                                 hidden_states=None                                         

In [48]:
import os

checkpoint_path = "finaltraining_1distillbert/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])



In [49]:

history=model.fit([X_train,train_mask],y_train,batch_size=32,epochs=8,validation_data=([X_test,test_mask],y_test),callbacks=[cp_callback])

Epoch 1/8
3354/3355 [============================>.] - ETA: 0s - loss: 0.3556 - accuracy: 0.8477
Epoch 1: saving model to finaltraining_1distillbert\cp.ckpt
3355/3355 [==============================] - 120s 35ms/step - loss: 0.3556 - accuracy: 0.8477 - val_loss: 0.2422 - val_accuracy: 0.9029
Epoch 2/8
3355/3355 [==============================] - ETA: 0s - loss: 0.2545 - accuracy: 0.8993
Epoch 2: saving model to finaltraining_1distillbert\cp.ckpt
3355/3355 [==============================] - 123s 37ms/step - loss: 0.2545 - accuracy: 0.8993 - val_loss: 0.2060 - val_accuracy: 0.9173
Epoch 3/8
3354/3355 [============================>.] - ETA: 0s - loss: 0.2227 - accuracy: 0.9143
Epoch 3: saving model to finaltraining_1distillbert\cp.ckpt
3355/3355 [==============================] - 121s 36ms/step - loss: 0.2227 - accuracy: 0.9143 - val_loss: 0.1909 - val_accuracy: 0.9227
Epoch 4/8
3354/3355 [============================>.] - ETA: 0s - loss: 0.2059 - accuracy: 0.9198
Epoch 4: saving model to

In [50]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [51]:


# Loads the weights
model.load_weights(checkpoint_path)


In [52]:
loss, acc = model.evaluate([X_test,test_mask],y_test, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

839/839 - 19s - loss: 0.1788 - accuracy: 0.9301 - 19s/epoch - 23ms/step
Restored model, accuracy: 93.01%


In [53]:


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict on the test set
y_pred_prob = model.predict([X_test,test_mask])

839/839 [==============================] - 22s 24ms/step


In [54]:
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate accuracy, precision, and recall
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")
print(f"F1-score: {f1}")

Test Accuracy: 0.9301415797317437
Test Precision: 0.9130132703397484
Test Recall: 0.9547369950595757
F1-score: 0.9334090989807152


In [55]:

preddf1 = pd.read_csv('ISOT_Fake.csv') ## Fake

preddf1['label'] = 0.0

preddf2 = pd.read_csv('ISOT_True.csv') ##Real

preddf2['label'] = 1.0

preddf1.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0.0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0.0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0.0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0.0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0.0


In [56]:
preddf2.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1.0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1.0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1.0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1.0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1.0


In [57]:

combine = pd.concat([preddf1, preddf2], ignore_index=True)

combine = combine.sample(frac=1, random_state=42).reset_index(drop=True)

combine.head()

,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",0.0
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",1.0
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",1.0
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",0.0
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",1.0


In [58]:


def clean_text(temp):
    temp=re.sub("@\S+", " ", temp)
    temp=re.sub("https*\S+", " ", temp)
    temp=re.sub("#\S+", " ", temp)
    temp=re.sub("\'\w+", '', temp)
    temp=re.sub(r'\w*\d+\w*', '', temp)
    temp=re.sub('\s{2,}', " ", temp)
    return temp.strip()


combine['tile'] = combine['title'].apply(clean_text)
combine['text_clean'] = combine['text'].apply(clean_text)
combine['sub'] = combine['subject'].apply(clean_text)
combine['dat'] = combine['date'].apply(clean_text)


#sentences3 = combine['tile'] + " " + combine['text_clean'] + " " + combine['sub'] + " " + combine['dat']
sentences3 = combine['tile'] + " " + combine['text_clean'] + " " + combine['sub']



In [59]:

input_ids3=[]
attention_masks3=[]


for sent in tqdm(sentences3):
    bert_inp=distilbert_tokenizer.encode_plus(sent,add_special_tokens = True,truncation=True,max_length =28,pad_to_max_length = True,return_attention_mask = True)
    input_ids3.append(bert_inp['input_ids'])
    attention_masks3.append(bert_inp['attention_mask'])

new_input_ids=np.asarray(input_ids3)
new_attention_mask=np.array(attention_masks3)
new_labels = np.array(combine['label'])



100%|████████████████████████████████████████████████████████████████████████████| 44898/44898 [08:21<00:00, 89.56it/s]


In [60]:

X_trainnew, X_testnew, y_trainnew, y_testnew,train_masknew,test_masknew=train_test_split(new_input_ids,new_labels,new_attention_mask,test_size=0.2,random_state=42)

In [61]:

checkpoint_path2 = "finaltraining_2distillbert/cp.ckpt"
checkpoint_dir2 = os.path.dirname(checkpoint_path2)

# Create a callback that saves the model's weights
cp_callback2 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path2,
                                                 save_weights_only=True,
                                                 verbose=1)



from tensorflow.keras.optimizers import Adam

# Recompile the model with a smaller learning rate
model.compile(
    optimizer=Adam(learning_rate=2e-5),  # Smaller learning rate for fine-tuning
    loss='binary_crossentropy',         # Same loss function as before
    metrics=['accuracy']                # Same evaluation metrics
)




In [62]:

# Fine-tuning with new data
history2 = model.fit(
    [X_trainnew, train_masknew],  # New input data
     y_trainnew,                           # New labels
    batch_size=32,
    epochs=7,  # Use fewer epochs for fine-tuning
    validation_data=([X_testnew, test_masknew], y_testnew),callbacks=[cp_callback2] 
)

Epoch 1/7
1122/1123 [============================>.] - ETA: 0s - loss: 0.7772 - accuracy: 0.6696
Epoch 1: saving model to finaltraining_2distillbert\cp.ckpt
1123/1123 [==============================] - 57s 43ms/step - loss: 0.7771 - accuracy: 0.6696 - val_loss: 0.4849 - val_accuracy: 0.7871
Epoch 2/7
1123/1123 [==============================] - ETA: 0s - loss: 0.3813 - accuracy: 0.8403
Epoch 2: saving model to finaltraining_2distillbert\cp.ckpt
1123/1123 [==============================] - 44s 39ms/step - loss: 0.3813 - accuracy: 0.8403 - val_loss: 0.2652 - val_accuracy: 0.8944
Epoch 3/7
1122/1123 [============================>.] - ETA: 0s - loss: 0.2288 - accuracy: 0.9132
Epoch 3: saving model to finaltraining_2distillbert\cp.ckpt
1123/1123 [==============================] - 42s 38ms/step - loss: 0.2287 - accuracy: 0.9132 - val_loss: 0.1666 - val_accuracy: 0.9354
Epoch 4/7
1122/1123 [============================>.] - ETA: 0s - loss: 0.1542 - accuracy: 0.9464
Epoch 4: saving model to fi

In [63]:


# Loads the weights
model.load_weights(checkpoint_path2)



In [64]:


loss, acc = model.evaluate([X_testnew, test_masknew], y_testnew, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))




281/281 - 6s - loss: 0.0404 - accuracy: 0.9863 - 6s/epoch - 23ms/step
Restored model, accuracy: 98.63%


In [65]:


# Predict on the test set
y_pred_prob2 = model.predict([X_testnew, test_masknew])


281/281 [==============================] - 11s 31ms/step


In [66]:


y_pred2 = (y_pred_prob2 > 0.5).astype(int)

# Calculate accuracy, precision, and recall
accuracy2 = accuracy_score(y_testnew, y_pred2)
precision2 = precision_score(y_testnew, y_pred2)
recall2 = recall_score(y_testnew, y_pred2)
f12 = f1_score(y_testnew, y_pred2)

print(f"Test Accuracy: {accuracy2}")
print(f"Test Precision: {precision2}")
print(f"Test Recall: {recall2}")
print(f"F1-score: {f12}")

Test Accuracy: 0.9863028953229399
Test Precision: 0.9841232780761149
Test Recall: 0.9871194379391101
F1-score: 0.985619081024202
